In [1]:
!pip install torch==2.2.1

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install gymnasium
!pip install gymnasium[classic-control]

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable

In [3]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random
import torch
from torch import nn
import torch.nn.functional as F


In [10]:

# Define model
class DQN(nn.Module):
    def __init__(self, in_states, h1_nodes, out_actions):
        super().__init__()

        # Define network layers
        self.fc1 = nn.Linear(in_states, h1_nodes)   # first fully connected layer
        self.out = nn.Linear(h1_nodes, out_actions) # ouptut layer w

    def forward(self, x):
        x = F.relu(self.fc1(x)) # Apply rectified linear unit (ReLU) activation
        #print(x)
        x = self.out(x)         # Calculate output
        return x

# Define memory for Experience Replay
class ReplayMemory():
    def __init__(self, maxlen):
        self.memory = deque([], maxlen=maxlen)

    def append(self, transition):
        self.memory.append(transition)

    def sample(self, sample_size):
        return random.sample(self.memory, sample_size)

    def __len__(self):
        return len(self.memory)

# MountainCar Deep Q-Learning
class MountainCarDQL():
    # Hyperparameters (adjustable)
    learning_rate_a = 0.01         # learning rate (alpha)
    discount_factor_g = 0.9         # discount rate (gamma)
    network_sync_rate = 50000          # number of steps the agent takes before syncing the policy and target network
    replay_memory_size = 100000       # size of replay memory
    mini_batch_size = 32            # size of the training data set sampled from the replay memory

    num_divisions = 20

    # Neural Network
    loss_fn = nn.MSELoss()          # NN Loss function. MSE=Mean Squared Error can be swapped to something else.
    optimizer = None                # NN Optimizer. Initialize later.


    # Train the environment
    def train(self, episodes, render=False):
        # Create FrozenLake instance
        env = gym.make('MountainCar-v0', render_mode='human' if render else None)
        num_states = env.observation_space.shape[0] # expecting 2: position & velocity
        num_actions = env.action_space.n

        # Divide position and velocity into segments
        self.pos_space = np.linspace(env.observation_space.low[0], env.observation_space.high[0], self.num_divisions)    # Between -1.2 and 0.6
        self.vel_space = np.linspace(env.observation_space.low[1], env.observation_space.high[1], self.num_divisions)    # Between -0.07 and 0.07

        epsilon = 1 # 1 = 100% random actions
        memory = ReplayMemory(self.replay_memory_size)

        # Create policy and target network. Number of nodes in the hidden layer can be adjusted.
        policy_dqn = DQN(in_states=num_states, h1_nodes=10, out_actions=num_actions)
        target_dqn = DQN(in_states=num_states, h1_nodes=10, out_actions=num_actions)

        # Make the target and policy networks the same (copy weights/biases from one network to the other)
        target_dqn.load_state_dict(policy_dqn.state_dict())

        # Policy network optimizer. "Adam" optimizer can be swapped to something else.
        self.optimizer = torch.optim.Adam(policy_dqn.parameters(), lr=self.learning_rate_a)

        # List to keep track of rewards collected per episode. Initialize list to 0's.
        rewards_per_episode = []

        # List to keep track of epsilon decay
        epsilon_history = []

        # Track number of steps taken. Used for syncing policy => target network.
        step_count=0
        goal_reached=False
        best_rewards=-200

        for i in range(episodes):
            state = env.reset()[0]  # Initialize to state 0
            terminated = False      # True when agent falls in hole or reached goal

            rewards = 0

            # Agent navigates map until it falls into hole/reaches goal (terminated), or has taken 200 actions (truncated).
            while(not terminated and rewards>-1000):

                # Select action based on epsilon-greedy
                if random.random() < epsilon:
                    # select random action
                    action = env.action_space.sample() # actions: 0=left,1=idle,2=right
                else:
                    # select best action
                    with torch.no_grad():
                        #print(policy_dqn(self.state_to_dqn_input(state)))
                        action = policy_dqn(self.state_to_dqn_input(state)).argmax().item()

                # Execute action
                new_state,reward,terminated,truncated,_ = env.step(action)

                # Accumulate reward
                rewards += reward

                # Save experience into memory
                memory.append((state, action, new_state, reward, terminated))

                # Move to the next state
                state = new_state

                # Increment step counter
                step_count+=1

            # Keep track of the rewards collected per episode.
            rewards_per_episode.append(rewards)
            if(terminated):
                goal_reached = True

            # Graph training progress
            if(i!=0 and i%1000==0):
                print(f'Episode {i} Epsilon {epsilon}')

                self.plot_progress(rewards_per_episode, epsilon_history)

            if rewards>best_rewards:
                best_rewards = rewards
                print(f'Best rewards so far: {best_rewards}')
                # Save policy
                torch.save(policy_dqn.state_dict(), f"mountaincar_dql_{i}.pt")

            # Check if enough experience has been collected
            if len(memory)>self.mini_batch_size:# and goal_reached:
                mini_batch = memory.sample(self.mini_batch_size)
                self.optimize(mini_batch, policy_dqn, target_dqn)

                # Decay epsilon
                epsilon = max(epsilon - 1/episodes, 0)
                epsilon_history.append(epsilon)

                # Copy policy network to target network after a certain number of steps
                if step_count > self.network_sync_rate:
                    target_dqn.load_state_dict(policy_dqn.state_dict())
                    step_count=0

        # Close environment
        env.close()
    def plot_progress(self, rewards_per_episode, epsilon_history):
        # Create new graph
        plt.figure(1)

        # Plot average rewards (Y-axis) vs episodes (X-axis)
        # rewards_curve = np.zeros(len(rewards_per_episode))
        # for x in range(len(rewards_per_episode)):
            # rewards_curve[x] = np.min(rewards_per_episode[max(0, x-10):(x+1)])
        plt.subplot(121) # plot on a 1 row x 2 col grid, at cell 1
        # plt.plot(sum_rewards)
        plt.plot(rewards_per_episode)

        # Plot epsilon decay (Y-axis) vs episodes (X-axis)
        plt.subplot(122) # plot on a 1 row x 2 col grid, at cell 2
        plt.plot(epsilon_history)

        # Save plots
        plt.savefig('mountaincar_dql.png')
    # Optimize policy network
    def optimize(self, mini_batch, policy_dqn, target_dqn):

        current_q_list = []
        target_q_list = []

        for state, action, new_state, reward, terminated in mini_batch:

            if terminated:
                # Agent receive reward of 0 for reaching goal.
                # When in a terminated state, target q value should be set to the reward.
                target = torch.FloatTensor([reward])
            else:
                # Calculate target q value
                with torch.no_grad():
                    target = torch.FloatTensor(
                        reward + self.discount_factor_g * target_dqn(self.state_to_dqn_input(new_state)).max()
                    )

            # Get the current set of Q values
            current_q = policy_dqn(self.state_to_dqn_input(state))
            current_q_list.append(current_q)

            # Get the target set of Q values
            target_q = target_dqn(self.state_to_dqn_input(state))
            # Adjust the specific action to the target that was just calculated
            target_q[action] = target
            target_q_list.append(target_q)

        # Compute loss for the whole minibatch
        loss = self.loss_fn(torch.stack(current_q_list), torch.stack(target_q_list))
        #print(loss)
        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    '''
    Converts a state (position, velocity) to tensor representation.
    Example:
    Input = (0.3, -0.03)
    Return = tensor([16, 6])
    '''
    def state_to_dqn_input(self, state)->torch.Tensor:
        state_p = np.digitize(state[0], self.pos_space)
        state_v = np.digitize(state[1], self.vel_space)

        return torch.FloatTensor([state_p, state_v])

    # Run the environment with the learned policy
    def test(self, episodes, model_filepath):
        # Create FrozenLake instance
        env = gym.make('MountainCar-v0', render_mode='human')
        num_states = env.observation_space.shape[0]
        num_actions = env.action_space.n

        self.pos_space = np.linspace(env.observation_space.low[0], env.observation_space.high[0], self.num_divisions)    # Between -1.2 and 0.6
        self.vel_space = np.linspace(env.observation_space.low[1], env.observation_space.high[1], self.num_divisions)    # Between -0.07 and 0.07

        # Load learned policy
        policy_dqn = DQN(in_states=num_states, h1_nodes=10, out_actions=num_actions)
        policy_dqn.load_state_dict(torch.load(model_filepath))
        policy_dqn.eval()    # switch model to evaluation mode

        for i in range(episodes):
            state = env.reset()[0]  # Initialize to state 0
            terminated = False      # True when agent falls in hole or reached goal
            truncated = False       # True when agent takes more than 200 actions

            # Agent navigates map until it falls into a hole (terminated), reaches goal (terminated), or has taken 200 actions (truncated).
            while(not terminated and not truncated):
                # Select best action
                with torch.no_grad():
                    var = policy_dqn(self.state_to_dqn_input(state))
                    print(var)
                    action = var.argmax().item()

                # Execute action
                state,reward,terminated,truncated,_ = env.step(action)

        env.close()

     # Run the environment with the learned policy
    def print(self, model_filepath):
       
        # Load learned policy
        policy_dqn = DQN(in_states=2, h1_nodes=10, out_actions=3)
        policy_dqn.load_state_dict(torch.load(model_filepath))
        print(policy_dqn)
        print(policy_dqn.fc1.weight, policy_dqn.fc1.bias) # = nn.Linear(in_states, h1_nodes)   # first fully connected layer
        print(policy_dqn.out.weight, policy_dqn.out.bias)


In [11]:
mountaincar = MountainCarDQL()

In [ ]:
mountaincar.train(25000, False)

In [9]:
mountaincar.test(1, "mountaincar_dql_23431.pt")

tensor([-9.7015, -9.7027, -9.7831])
tensor([-9.7015, -9.7027, -9.7831])
tensor([-9.7015, -9.7027, -9.7831])
tensor([-9.7843, -9.7895, -9.9375])
tensor([-9.7843, -9.7895, -9.9375])
tensor([-9.7843, -9.7895, -9.9375])
tensor([-9.7843, -9.7895, -9.9375])
tensor([-9.7843, -9.7895, -9.9375])
tensor([-9.7843, -9.7895, -9.9375])
tensor([ -9.8672,  -9.8762, -10.0918])
tensor([ -9.8672,  -9.8762, -10.0918])
tensor([ -9.9524,  -9.9626, -10.1937])
tensor([ -9.9524,  -9.9626, -10.1937])
tensor([ -9.9524,  -9.9626, -10.1937])
tensor([ -9.9524,  -9.9626, -10.1937])
tensor([ -9.9524,  -9.9626, -10.1937])
tensor([ -9.9524,  -9.9626, -10.1937])
tensor([-10.0375, -10.0490, -10.2956])
tensor([-10.0375, -10.0490, -10.2956])
tensor([-10.0375, -10.0490, -10.2956])
tensor([-10.0375, -10.0490, -10.2956])
tensor([-10.0375, -10.0490, -10.2956])
tensor([-10.1226, -10.1354, -10.3975])
tensor([-10.1226, -10.1354, -10.3975])
tensor([-10.1226, -10.1354, -10.3975])
tensor([-10.1226, -10.1354, -10.3975])
tensor([-10.1

In [12]:
mountaincar.print("mountaincar_dql_23431.pt")

DQN(
  (fc1): Linear(in_features=2, out_features=10, bias=True)
  (out): Linear(in_features=10, out_features=3, bias=True)
)
Parameter containing:
tensor([[-0.1692, -0.5422],
        [-0.5403, -0.1118],
        [-0.7500, -0.0213],
        [-0.7580,  0.0218],
        [ 0.0900,  0.5659],
        [ 0.0203,  0.6060],
        [-0.2351, -0.3665],
        [-0.0348, -0.7156],
        [-0.1158, -0.5073],
        [-0.5872,  0.0215]], requires_grad=True) Parameter containing:
tensor([-0.0757, -0.6626, -0.3756,  0.2973, -3.4696,  4.5481, -0.5467, -0.2575,
         0.6185,  0.1723], requires_grad=True)
Parameter containing:
tensor([[-0.3058,  0.2379, -0.0113, -0.1135,  1.1597, -0.9462, -0.0374,  0.2266,
         -0.2753, -0.2412],
        [ 0.1222,  0.1303,  0.1516,  0.0678,  1.1755, -0.9546, -0.1078,  0.0779,
         -0.0402,  0.0219],
        [-0.2718,  0.3103,  0.1038,  0.0866,  1.3623, -1.0175, -0.0760,  0.2576,
         -0.1153,  0.0337]], requires_grad=True) Parameter containing:
tensor([-2.